# 네이버 포탈 카페 크롤링 

## 수정 내용
1. **버그 수정**: `keywords` → `key` (275번 줄 - 전체 리스트가 아닌 현재 키워드 저장)
2. **안정성 개선**: iframe 전환 전 default_content로 복귀
3. **에러 처리 강화**: 탭 핸들링 로직 개선
4. **무한 루프 방지**: 최대 페이지 수 제한 추가

In [100]:
# 필요시 패키지 설치 (주석 해제 후 실행)
# !pip install selenium
# !pip install beautifulsoup4
# !pip install lxml
# !pip install pandas
# !pip install tqdm

In [101]:
import logging
import random
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, 
    NoSuchElementException, 
    NoAlertPresentException,
    StaleElementReferenceException,
    WebDriverException
)
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings(action='ignore')

# 로그 설정
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print("라이브러리 로드 완료!")

라이브러리 로드 완료!


## 1. 설정
### 1.1 키워드 설정

In [102]:
# 메인 키워드 설정
main_keywords = ['에어컨']

# 에어컨 세부 키워드
aircond_key = ['약풍']

eliminate_ads_keyword = ''

### 1.2 제외할 카페 설정

In [103]:
# 제외할 카페 목록 (필수 실행)
eliminate_cafe = [
    'https://cafe.naver.com/allfm01',
    'https://cafe.naver.com/autowave21',
    'https://cafe.naver.com/ipod5',
    'https://cafe.naver.com/saejaeserver/5179',
    'https://cafe.naver.com/clubkiakue',
    'https://cafe.naver.com/xxyxx',
    'https://cafe.naver.com/story77',
    'https://cafe.naver.com/bongo312',
    'https://cafe.naver.com/hyun6587',
    'https://cafe.naver.com/jihosoccer123',
    'https://cafe.naver.com/lexusclubkorea',
    'https://cafe.naver.com/haruhi',
    'https://cafe.naver.com/old13',
    'https://cafe.naver.com/mp3vilge',
    'https://cafe.naver.com/pantagi',
    'https://cafe.naver.com/ioniq9owners',
    'https://cafe.naver.com/gpsf',
    'https://cafe.naver.com/viviv',
    'https://cafe.naver.com/bananawork',
    'https://cafe.naver.com/volvocrew',
    'https://cafe.naver.com/esmartwoman',
    'https://cafe.naver.com/silver0989',
    'https://cafe.naver.com/kig',
    'https://cafe.naver.com/mycar2010',
    'https://cafe.naver.com/gruu',
    'http://cafe.naver.com/joonggonara',
    'https://cafe.naver.com/campingfirst',
    'https://cafe.naver.com/imsanbu',
    'https://cafe.naver.com/remonterrace',
    'https://cafe.naver.com/zzop',
    'https://cafe.naver.com/astonishiacafe',
    ' https://cafe.naver.com/mindy7857'
]

print(f"제외할 카페: {len(eliminate_cafe)}개")

제외할 카페: 32개


### 1.3 키워드 조합 생성

In [104]:
# 키워드 조합 생성
keywords = []

for main in main_keywords:
    for air in aircond_key:
        keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

print("생성된 키워드:")
for i, kw in enumerate(keywords, 1):
    print(f"  {i}. {kw}")

생성된 키워드:
  1. 에어컨+%2B약풍 


## 2. 헬퍼 함수 정의

In [105]:
def safe_close_extra_tabs(driver, keep_count=1):
    """
    안전하게 추가 탭들을 닫는 함수
    
    Args:
        driver: Selenium WebDriver
        keep_count: 유지할 탭 수 (기본값: 1)
    """
    try:
        max_attempts = 10  # 무한 루프 방지
        attempts = 0
        
        while len(driver.window_handles) > keep_count and attempts < max_attempts:
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            time.sleep(0.3)
            attempts += 1
        
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        logging.warning(f"탭 닫기 중 오류: {e}")
        try:
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass


def handle_alert(driver, timeout=2):
    """
    Alert 팝업 처리 함수
    
    Args:
        driver: Selenium WebDriver
        timeout: 대기 시간 (초)
    
    Returns:
        bool: Alert가 있었으면 True, 없었으면 False
    """
    try:
        WebDriverWait(driver, timeout).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert_text = alert.text
        logging.info(f"Alert 감지: {alert_text}")
        alert.accept()
        return True
    except (TimeoutException, NoAlertPresentException):
        return False


def switch_to_cafe_frame(driver, max_retries=3):
    """
    카페 메인 프레임으로 전환하는 함수
    
    Args:
        driver: Selenium WebDriver
        max_retries: 최대 재시도 횟수
    
    Returns:
        bool: 성공 여부
    """
    for attempt in range(max_retries):
        try:
            # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
            driver.switch_to.default_content()
            time.sleep(0.5)
            
            # iframe 존재 확인
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'cafe_main'))
            )
            
            # 프레임 전환
            driver.switch_to.frame('cafe_main')
            return True
        except Exception as e:
            logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
            if attempt < max_retries - 1:
                driver.refresh()
                time.sleep(2)
    return False


def extract_post_data(driver):
    """
    게시물 데이터 추출 함수
    
    Args:
        driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
    Returns:
        dict or None: 추출된 데이터 또는 실패시 None
    """
    try:
        soup = bs(driver.page_source, 'lxml')
        
        # 제목 추출
        title_elem = soup.select_one('h3.title_text')
        if not title_elem:
            return None
        title = title_elem.text.strip()
        
        # 본문 추출
        contents = [i.text for i in soup.select('div.se-component-content')]
        if len(contents) == 0:
            contents = [i.text for i in soup.select('div.ContentRenderer')]
        contents = ' '.join(contents)
        contents = contents.replace('\u200b', ' ')
        contents = re.sub(r'\s+', ' ', contents).strip()
        
        # 날짜 추출
        date_elem = soup.select_one('span.date')
        date = date_elem.text.strip() if date_elem else ''
        
        # 댓글 추출
        comments = [i.text for i in soup.select('span.text_comment')]
        
        # 글쓴이 닉네임 추출
        try:
            author_nick_elem = soup.select_one('button.nickname')
            if author_nick_elem:
                author_nickname = author_nick_elem.text.strip()
            else:
                # WriterInfo 영역에서 찾기
                writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
                author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
        except:
            author_nickname = '닉네임 없음'
        
        # 댓글 작성자 닉네임 추출
        try:
            comment_nick_elems = soup.select('a.comment_nickname')
            if comment_nick_elems:
                comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
            else:
                comment_nicknames = '댓글 닉네임 없음'
        except:
            comment_nicknames = '댓글 닉네임 없음'
        
        return {
            'title': title,
            'contents': contents,
            'date': date,
            'comments': comments,
            'author_nickname': author_nickname,
            'comment_nicknames': comment_nicknames
        }
    except Exception as e:
        logging.error(f"데이터 추출 오류: {e}")
        return None


print("헬퍼 함수 정의 완료!")

헬퍼 함수 정의 완료!


## 3. WebDriver 설정 및 실행

In [106]:
# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# options.add_argument('--headless')

# WebDriver 실행
try:
    driver = webdriver.Chrome(options=options)
    print("WebDriver 실행 성공!")
except WebDriverException as e:
    print(f"WebDriver 실행 실패: {e}")
    print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")

WebDriver 실행 성공!


In [107]:
# 데이터프레임 초기화
dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
index = 0

print("데이터프레임 초기화 완료!")

데이터프레임 초기화 완료!


## 4. 크롤링 실행

⚠️ **주의사항**
- 진짜,,오래 걸려요ㅜㅜㅜㅜㅜㅜ 한번에 많이 크롤링 하기 위한,,어쩔수 없는 방법이였습니다,,모두들 화이팅,,,


In [108]:

TARGET_COUNT = 3000  
MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


YEAR_RANGES = [ 
    ("20250101", "20251031"),  
    ("20240101", "20241231"),
    ("20230101", "20231231"), 
    ("20220101", "20221231"),  
    ("20210101", "20211231")  
]

# 전체 진행률 바 생성
total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
current_collected = 0

for key in keywords:
    if current_collected >= TARGET_COUNT:
        break
        
    logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
    # 설정한 날짜 구간별로 루프 실행
    for start_date_str, end_date_str in YEAR_RANGES:
        if current_collected >= TARGET_COUNT:
            break

        logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
        # 검색 URL 생성
        search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
        driver.get(search_url)
        driver.implicitly_wait(15)
        time.sleep(2)
        
        breaker = False
        start_n, end_n = 1, 31
        consecutive_errors = 0
        
        # 한 구간 내에서 페이지 넘김
        while not breaker:
            # 목표 달성 시 루프 탈출
            if len(dataframe) >= TARGET_COUNT:
                breaker = True
                break

            # 현재 로딩된 게시물 수 확인
            check_soup = bs(driver.page_source, 'lxml')
            post_elements = check_soup.select('div.title_area')
            check_len = len(post_elements)
            
            # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
            if check_len < start_n:
                loading_success = False
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2.5)
                    
                    check_soup = bs(driver.page_source, 'lxml')
                    if len(check_soup.select('div.title_area')) >= start_n:
                        check_len = len(check_soup.select('div.title_area'))
                        loading_success = True
                        break
                
                if not loading_success:
                    logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
                    breaker = True
                    break

            # 게시물 하나씩 순회
            for i in range(start_n, end_n):
                if len(dataframe) >= TARGET_COUNT:
                    breaker = True
                    break

                if i > check_len:
                    break
                
                xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
                try:
                    # 요소 찾기
                    site = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, xpath))
                    )
                    href_value = site.get_attribute("href")
                    
                    # 제외 카페 필터링
                    if any(element in href_value for element in eliminate_cafe):
                        continue
                    
                    # 탭 열기
                    driver.execute_script(f"window.open('{href_value}', '_blank');")
                    time.sleep(random.uniform(0.5, 0.8))
                    
                    # 탭 전환
                    try:
                        WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
                        driver.switch_to.window(driver.window_handles[-1])
                    except:
                        continue 

                    time.sleep(random.uniform(0.3, 0.6))
                    
                    # 팝업 처리
                    if handle_alert(driver):
                        safe_close_extra_tabs(driver)
                        continue
                    
                    # 데이터 추출
                    if switch_to_cafe_frame(driver):
                        post_data = extract_post_data(driver)
                        
                        if post_data and post_data['title']:
                            current_url = driver.current_url
                            
                            # 중복 방지
                            if not dataframe['url'].isin([current_url]).any():
                                input_data = [
                                    post_data['date'],
                                    key,
                                    post_data['title'],
                                    post_data['contents'],
                                    post_data['comments'],
                                    '네이버포탈',
                                    current_url,
                                    post_data['author_nickname'],
                                    post_data['comment_nicknames']
                                ]
                                dataframe.loc[index] = input_data
                                index += 1
                                current_collected += 1
                                total_pbar.update(1)
                                consecutive_errors = 0 
                            else:
                                logging.info("중복 URL 발견, 스킵")
                        else:
                            consecutive_errors += 1
                    else:
                        consecutive_errors += 1
                    
                    # 탭 닫기
                    safe_close_extra_tabs(driver)
                    
                    # 연속 에러 체크
                    if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
                        logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
                        breaker = True
                        break

                except Exception as e:
                    safe_close_extra_tabs(driver)
                    continue

            # 다음 페이지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            start_n += 30
            end_n += 30

    # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
    if current_collected >= TARGET_COUNT:
        break

total_pbar.close()
print(f"\n=============================================")
print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
print(f"=============================================")

전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-15 00:17:54,719 - INFO - ========== [에어컨+%2B약풍 ] 키워드 수집 시작 ==========
2025-12-15 00:17:54,720 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-15 00:46:42,413 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 453개). 다음 구간으로 이동.
2025-12-15 00:46:42,414 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-15 01:30:07,010 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 660개). 다음 구간으로 이동.
2025-12-15 01:30:07,011 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-15 01:48:06,635 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 316개). 다음 구간으로 이동.
2025-12-15 01:48:06,636 - INFO - 검색 구간: 20220101 ~ 20221231
2025-12-15 02:08:01,306 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 311개). 다음 구간으로 이동.
2025-12-15 02:08:01,307 - INFO - 검색 구간: 20210101 ~ 20211231
2025-12-15 02:27:48,905 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 300개). 다음 구간으로 이동.



크롤링 최종 완료! 총 1811개 게시물 수집됨


## 5. 결과 확인 및 저장

In [109]:
# 데이터 확인
print(f"수집된 총 게시물 수: {len(dataframe)}")
dataframe.head(100)

수집된 총 게시물 수: 1811


,date,keyword,title,contents,comments,site,url,author_nickname,comment_nicknames
0,2025.08.06. 08:54,에어컨+%2B약풍,지하철에서 에어컨 약풍..,사람 꽉 차있는데 에어컨 약풍으로 틀어놔서진짜 땀 뻘뻘 흘렸어요ㅠㅠ,[],네이버포탈,https://cafe.naver.com/knife67/384180?art=ZXh0...,해달의보물,댓글 닉네임 없음
1,2025.09.22. 16:51,에어컨+%2B약풍,25년식 523d 주행기(2),어쩌다보니 센터 작업이 좀 더 걸려서 대차로 받은 523d를 몇일 더 타게 되었습니...,"[이전글 보고 왔습니다. (1,2편 정주행)18d 11년, 20d 7년 ISG 항...",네이버포탈,https://cafe.naver.com/bimmerwerks/1065506?art...,RAY벤,shutterholic | RAY벤 | 모던심플히든
2,2025.09.08. 05:55,에어컨+%2B약풍,갑자기 가을 아침이 되었어요,아파트 입구를 나서는데에어컨 약풍같은 서늘한 기온이 느껴지더라구요 반팔은 춥네요 덥...,"[그러게요 날씨가 가을이 온거같아요, 가을이 기대되요 :)]",네이버포탈,https://cafe.naver.com/segokmoa/100137?art=ZXh...,썬라이트,보보려잉 | 썬라이트
3,2025.06.13. 15:19,에어컨+%2B약풍,에어컨 25도무풍or 26도 약풍,오늘 에어컨 삼성무풍 설치받아서 테스트중인데요26도 무풍으로하니 그 꿉꿉한냄시 나더...,"[무풍이냄새나서 무풍기능안썻어요, 25무풍은 냄새 안나는디 26무풍은 왜 냄새가 날...",네이버포탈,https://cafe.naver.com/magic26/2223994?art=ZXh...,호미,날마다행복해 | 호미
4,2025.07.10. 13:54,에어컨+%2B약풍,이제 에어컨 안틀고 자면 너무 덥네요,이젠 에어컨 끄고 못 자겠어요 ㅋㅋ아침에 깨면 더워서 땀 흘리면서 깨네요에어컨 약풍...,"[조금 덜쓰고...에어컨틀고 잘래요ㅠㅠ, 저는 더위를 너무 많이 타서ㅜㅜ 자고 일어...",네이버포탈,https://cafe.naver.com/2myloft/394776?art=ZXh0...,가지챙경채,껌딱지 | 가지챙경채 | 댜우니 | 가지챙경채 | I몽상가I | 가지챙경채 | I몽...
...,...,...,...,...,...,...,...,...,...
95,2025.10.14. 13:17,에어컨+%2B약풍,(1등 당첨 후기) 여름이 이제 반가워질 것 같아요💙 비스포크 AI 무풍콤보 갤러리...,"지금 나도 Bespoke AI와 삽니다🎉7월 이벤트 1등 수상자의 리얼 후기, 지금...",[오이선인장 님 집안 분위기를 더 세련되게 바꾸어준비스포크 AI 무풍콤보 갤러리 에...,네이버포탈,https://cafe.naver.com/mjann/3685024?art=ZXh0Z...,비스포크AI가전이벤트,친절한 세일맘
96,2025.06.02. 14:51,에어컨+%2B약풍,여름에 4일 집을 비우게 됐는데 에어컨 틀고 나가면 되겠죠?,7월이라 아무래도 날씨가 제일 걱정이네요ㅠㅠ26도 약풍으로 맞추고 나가도 되겠죠.....,[물 밥 온도만 맞으면 혼자 잘 있을거에요],네이버포탈,https://cafe.naver.com/myfriendspet/2166140?ar...,삐코삐,엄살이집사
97,2025.07.29. 16:47,에어컨+%2B약풍,한시간 걷고왔더니..,"어제보다 더 덥네요.샤워하고,선풍기돌리고,암막커튼치고 휴식취하고있어요.에어컨바람싫어...","[실내가 최곱니다,,,, 그래서 밥먹을때만나오고, 각자 방으로 들어가서 안나와요.ㅠ...",네이버포탈,https://cafe.naver.com/babylovecafe/1116052?ar...,태양 별,이웃집도토리묵 | 태양 별 | 심플리라이프 | 불친절한지우씨 | 태양 별 | 불친절...
98,2025.07.18. 15:39,에어컨+%2B약풍,LG 인버터 에어컨 절전모드 설정 썰....,구매한지 8년쯤 된 구형입니다.(요즘 신형은 어떨지는 몰라요;;;) LG 인버터 에...,[],네이버포탈,https://cafe.naver.com/koreassistant/21181?art...,어쩌다보니둘기가된,댓글 닉네임 없음


In [110]:
# URL 기준 중복 제거
dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
print(f"중복 제거 후 게시물 수: {len(dataframe)}")

중복 제거 후 게시물 수: 1811


In [111]:
ad_keywords = [
 '매도', '임대', '계약', '가이드','팝니다' 
    '업체', '이벤트', '대여', '앵콜', '비트코인', 
    '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
]

def is_ad_post(row):
    text = str(row['title']) + ' ' + str(row['contents'])
    return any(kw in text for kw in ad_keywords)

In [112]:
print(f"필터링 전: {len(dataframe)}개")
dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
print(f"필터링 후: {len(dataframe)}개")

필터링 전: 1811개
필터링 후: 1570개


In [113]:
csv_filename = "네이버포탈_크롤링결과_약풍.csv"
dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)

In [114]:
# WebDriver 종료
driver.quit()
print("WebDriver 종료 완료!")

WebDriver 종료 완료!


In [115]:


# 로그 설정
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

print("라이브러리 로드 완료!")
## 1. 설정
### 1.1 키워드 설정
# 메인 키워드 설정
main_keywords = ['에어컨']

# 에어컨 세부 키워드
aircond_key = ['열대야']

eliminate_ads_keyword = ''
### 1.2 제외할 카페 설정
# 제외할 카페 목록 (필수 실행)
eliminate_cafe = [
    'https://cafe.naver.com/allfm01',
    'https://cafe.naver.com/autowave21',
    'https://cafe.naver.com/ipod5',
    'https://cafe.naver.com/saejaeserver/5179',
    'https://cafe.naver.com/clubkiakue',
    'https://cafe.naver.com/xxyxx',
    'https://cafe.naver.com/story77',
    'https://cafe.naver.com/bongo312',
    'https://cafe.naver.com/hyun6587',
    'https://cafe.naver.com/jihosoccer123',
    'https://cafe.naver.com/lexusclubkorea',
    'https://cafe.naver.com/haruhi',
    'https://cafe.naver.com/old13',
    'https://cafe.naver.com/mp3vilge',
    'https://cafe.naver.com/pantagi',
    'https://cafe.naver.com/ioniq9owners',
    'https://cafe.naver.com/gpsf',
    'https://cafe.naver.com/viviv',
    'https://cafe.naver.com/bananawork',
    'https://cafe.naver.com/volvocrew',
    'https://cafe.naver.com/esmartwoman',
    'https://cafe.naver.com/silver0989',
    'https://cafe.naver.com/kig',
    'https://cafe.naver.com/mycar2010',
    'https://cafe.naver.com/gruu',
    'http://cafe.naver.com/joonggonara',
    'https://cafe.naver.com/campingfirst',
    'https://cafe.naver.com/imsanbu',
    'https://cafe.naver.com/remonterrace',
    'https://cafe.naver.com/zzop',
    'https://cafe.naver.com/astonishiacafe',
    ' https://cafe.naver.com/mindy7857'
]

print(f"제외할 카페: {len(eliminate_cafe)}개")
### 1.3 키워드 조합 생성
# 키워드 조합 생성
keywords = []

for main in main_keywords:
    for air in aircond_key:
        keywords.append(main + '+%2B' + air + ' ' + eliminate_ads_keyword)

print("생성된 키워드:")
for i, kw in enumerate(keywords, 1):
    print(f"  {i}. {kw}")
## 2. 헬퍼 함수 정의
def safe_close_extra_tabs(driver, keep_count=1):
    """
    안전하게 추가 탭들을 닫는 함수
    
    Args:
        driver: Selenium WebDriver
        keep_count: 유지할 탭 수 (기본값: 1)
    """
    try:
        max_attempts = 10  # 무한 루프 방지
        attempts = 0
        
        while len(driver.window_handles) > keep_count and attempts < max_attempts:
            driver.switch_to.window(driver.window_handles[-1])
            driver.close()
            time.sleep(0.3)
            attempts += 1
        
        driver.switch_to.window(driver.window_handles[0])
    except Exception as e:
        logging.warning(f"탭 닫기 중 오류: {e}")
        try:
            driver.switch_to.window(driver.window_handles[0])
        except:
            pass


def handle_alert(driver, timeout=2):
    """
    Alert 팝업 처리 함수
    
    Args:
        driver: Selenium WebDriver
        timeout: 대기 시간 (초)
    
    Returns:
        bool: Alert가 있었으면 True, 없었으면 False
    """
    try:
        WebDriverWait(driver, timeout).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert_text = alert.text
        logging.info(f"Alert 감지: {alert_text}")
        alert.accept()
        return True
    except (TimeoutException, NoAlertPresentException):
        return False


def switch_to_cafe_frame(driver, max_retries=3):
    """
    카페 메인 프레임으로 전환하는 함수
    
    Args:
        driver: Selenium WebDriver
        max_retries: 최대 재시도 횟수
    
    Returns:
        bool: 성공 여부
    """
    for attempt in range(max_retries):
        try:
            # ★★★ 중요: 먼저 default content로 돌아가기 ★★★
            driver.switch_to.default_content()
            time.sleep(0.5)
            
            # iframe 존재 확인
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'cafe_main'))
            )
            
            # 프레임 전환
            driver.switch_to.frame('cafe_main')
            return True
        except Exception as e:
            logging.warning(f"프레임 전환 시도 {attempt + 1}/{max_retries} 실패: {e}")
            if attempt < max_retries - 1:
                driver.refresh()
                time.sleep(2)
    return False


def extract_post_data(driver):
    """
    게시물 데이터 추출 함수
    
    Args:
        driver: Selenium WebDriver (이미 cafe_main 프레임 내에 있어야 함)
    
    Returns:
        dict or None: 추출된 데이터 또는 실패시 None
    """
    try:
        soup = bs(driver.page_source, 'lxml')
        
        # 제목 추출
        title_elem = soup.select_one('h3.title_text')
        if not title_elem:
            return None
        title = title_elem.text.strip()
        
        # 본문 추출
        contents = [i.text for i in soup.select('div.se-component-content')]
        if len(contents) == 0:
            contents = [i.text for i in soup.select('div.ContentRenderer')]
        contents = ' '.join(contents)
        contents = contents.replace('\u200b', ' ')
        contents = re.sub(r'\s+', ' ', contents).strip()
        
        # 날짜 추출
        date_elem = soup.select_one('span.date')
        date = date_elem.text.strip() if date_elem else ''
        
        # 댓글 추출
        comments = [i.text for i in soup.select('span.text_comment')]
        
        # 글쓴이 닉네임 추출
        try:
            author_nick_elem = soup.select_one('button.nickname')
            if author_nick_elem:
                author_nickname = author_nick_elem.text.strip()
            else:
                # WriterInfo 영역에서 찾기
                writer_info = soup.select_one('div.WriterInfo a.nickname, div.WriterInfo button.nickname')
                author_nickname = writer_info.text.strip() if writer_info else '닉네임 없음'
        except:
            author_nickname = '닉네임 없음'
        
        # 댓글 작성자 닉네임 추출
        try:
            comment_nick_elems = soup.select('a.comment_nickname')
            if comment_nick_elems:
                comment_nicknames = ' | '.join([n.text.strip() for n in comment_nick_elems])
            else:
                comment_nicknames = '댓글 닉네임 없음'
        except:
            comment_nicknames = '댓글 닉네임 없음'
        
        return {
            'title': title,
            'contents': contents,
            'date': date,
            'comments': comments,
            'author_nickname': author_nickname,
            'comment_nicknames': comment_nicknames
        }
    except Exception as e:
        logging.error(f"데이터 추출 오류: {e}")
        return None


print("헬퍼 함수 정의 완료!")
## 3. WebDriver 설정 및 실행
# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Headless 모드 (화면 없이 실행) - 필요시 주석 해제
# options.add_argument('--headless')

# WebDriver 실행
try:
    driver = webdriver.Chrome(options=options)
    print("WebDriver 실행 성공!")
except WebDriverException as e:
    print(f"WebDriver 실행 실패: {e}")
    print("ChromeDriver가 설치되어 있고 Chrome 브라우저 버전과 일치하는지 확인하세요.")
# 데이터프레임 초기화
dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments', 'site', 'url', 'author_nickname', 'comment_nicknames'])
index = 0

print("데이터프레임 초기화 완료!")
## 4. 크롤링 실행

TARGET_COUNT = 3000  
MAX_CONSECUTIVE_ERRORS = 30  # 에러가 30번 연속 나면 다음 구간으로 점프


YEAR_RANGES = [ 
    ("20250101", "20251031"),  
    ("20240101", "20241231"),
    ("20230101", "20231231"), 
    ("20220101", "20221231"),  
    ("20210101", "20211231")  
]

# 전체 진행률 바 생성
total_pbar = tqdm(total=TARGET_COUNT, desc="전체 목표 달성률")
current_collected = 0

for key in keywords:
    if current_collected >= TARGET_COUNT:
        break
        
    logging.info(f"========== [{key}] 키워드 수집 시작 ==========")
    
    # 설정한 날짜 구간별로 루프 실행
    for start_date_str, end_date_str in YEAR_RANGES:
        if current_collected >= TARGET_COUNT:
            break

        logging.info(f"검색 구간: {start_date_str} ~ {end_date_str}")
        
        # 검색 URL 생성
        search_url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={key}&nso=so:r,p:from{start_date_str}to{end_date_str}'
        
        driver.get(search_url)
        driver.implicitly_wait(15)
        time.sleep(2)
        
        breaker = False
        start_n, end_n = 1, 31
        consecutive_errors = 0
        
        # 한 구간 내에서 페이지 넘김
        while not breaker:
            # 목표 달성 시 루프 탈출
            if len(dataframe) >= TARGET_COUNT:
                breaker = True
                break

            # 현재 로딩된 게시물 수 확인
            check_soup = bs(driver.page_source, 'lxml')
            post_elements = check_soup.select('div.title_area')
            check_len = len(post_elements)
            
            # 스크롤이 덜 돼서 게시물이 안 보이면 재시도 (최대 3회)
            if check_len < start_n:
                loading_success = False
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2.5)
                    
                    check_soup = bs(driver.page_source, 'lxml')
                    if len(check_soup.select('div.title_area')) >= start_n:
                        check_len = len(check_soup.select('div.title_area'))
                        loading_success = True
                        break
                
                if not loading_success:
                    logging.info(f"더 이상 게시글이 로딩되지 않음 (현재 {check_len}개). 다음 구간으로 이동.")
                    breaker = True
                    break

            # 게시물 하나씩 순회
            for i in range(start_n, end_n):
                if len(dataframe) >= TARGET_COUNT:
                    breaker = True
                    break

                if i > check_len:
                    break
                
                xpath = f'//*[@id="main_pack"]/section/div[1]/ul/li[{i}]/div/div[2]/div[2]/a'
                
                try:
                    # 요소 찾기
                    site = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.XPATH, xpath))
                    )
                    href_value = site.get_attribute("href")
                    
                    # 제외 카페 필터링
                    if any(element in href_value for element in eliminate_cafe):
                        continue
                    
                    # 탭 열기
                    driver.execute_script(f"window.open('{href_value}', '_blank');")
                    time.sleep(random.uniform(0.5, 0.8))
                    
                    # 탭 전환
                    try:
                        WebDriverWait(driver, 5).until(lambda d: len(d.window_handles) > 1)
                        driver.switch_to.window(driver.window_handles[-1])
                    except:
                        continue 

                    time.sleep(random.uniform(0.3, 0.6))
                    
                    # 팝업 처리
                    if handle_alert(driver):
                        safe_close_extra_tabs(driver)
                        continue
                    
                    # 데이터 추출
                    if switch_to_cafe_frame(driver):
                        post_data = extract_post_data(driver)
                        
                        if post_data and post_data['title']:
                            current_url = driver.current_url
                            
                            # 중복 방지
                            if not dataframe['url'].isin([current_url]).any():
                                input_data = [
                                    post_data['date'],
                                    key,
                                    post_data['title'],
                                    post_data['contents'],
                                    post_data['comments'],
                                    '네이버포탈',
                                    current_url,
                                    post_data['author_nickname'],
                                    post_data['comment_nicknames']
                                ]
                                dataframe.loc[index] = input_data
                                index += 1
                                current_collected += 1
                                total_pbar.update(1)
                                consecutive_errors = 0 
                            else:
                                logging.info("중복 URL 발견, 스킵")
                        else:
                            consecutive_errors += 1
                    else:
                        consecutive_errors += 1
                    
                    # 탭 닫기
                    safe_close_extra_tabs(driver)
                    
                    # 연속 에러 체크
                    if consecutive_errors >= MAX_CONSECUTIVE_ERRORS:
                        logging.warning(f"연속 에러 {consecutive_errors}회. 다음 구간으로 이동.")
                        breaker = True
                        break

                except Exception as e:
                    safe_close_extra_tabs(driver)
                    continue

            # 다음 페이지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            start_n += 30
            end_n += 30

    # 목표 달성 확인 및 종료 (로그 메시지 삭제됨)
    if current_collected >= TARGET_COUNT:
        break

total_pbar.close()
print(f"\n=============================================")
print(f"크롤링 최종 완료! 총 {len(dataframe)}개 게시물 수집됨")
print(f"=============================================")
## 5. 결과 확인 및 저장
# 데이터 확인
print(f"수집된 총 게시물 수: {len(dataframe)}")
dataframe.head(100)
# URL 기준 중복 제거
dataframe = dataframe.drop_duplicates(subset='url', keep='first').reset_index(drop=True)
print(f"중복 제거 후 게시물 수: {len(dataframe)}")
ad_keywords = [
 '매도', '임대', '계약', '가이드','팝니다' 
    '업체', '이벤트', '대여', '앵콜', '비트코인', 
    '초특가', '파격', '땡처리','고시원','매매','나눔','캠프','엔진','터보','시동','베트남'
]

def is_ad_post(row):
    text = str(row['title']) + ' ' + str(row['contents'])
    return any(kw in text for kw in ad_keywords)
print(f"필터링 전: {len(dataframe)}개")
dataframe = dataframe[~dataframe.apply(is_ad_post, axis=1)].reset_index(drop=True)
print(f"필터링 후: {len(dataframe)}개")
csv_filename = "네이버포탈_크롤링결과_열대야.csv"
dataframe.to_csv(csv_filename, encoding="utf-8-sig", index=False)
# WebDriver 종료
driver.quit()
print("WebDriver 종료 완료!")



라이브러리 로드 완료!
제외할 카페: 32개
생성된 키워드:
  1. 에어컨+%2B열대야 
헬퍼 함수 정의 완료!
WebDriver 실행 성공!
데이터프레임 초기화 완료!


전체 목표 달성률:   0%|          | 0/3000 [00:00<?, ?it/s]

2025-12-15 02:28:01,966 - INFO - ========== [에어컨+%2B열대야 ] 키워드 수집 시작 ==========
2025-12-15 02:28:01,967 - INFO - 검색 구간: 20250101 ~ 20251031
2025-12-15 03:34:41,786 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-15 03:34:41,787 - INFO - 검색 구간: 20240101 ~ 20241231
2025-12-15 04:40:15,822 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-15 04:40:15,823 - INFO - 검색 구간: 20230101 ~ 20231231
2025-12-15 05:47:28,602 - INFO - 더 이상 게시글이 로딩되지 않음 (현재 1020개). 다음 구간으로 이동.
2025-12-15 05:47:28,603 - INFO - 검색 구간: 20220101 ~ 20221231



크롤링 최종 완료! 총 3000개 게시물 수집됨
수집된 총 게시물 수: 3000
중복 제거 후 게시물 수: 3000
필터링 전: 3000개
필터링 후: 2863개
WebDriver 종료 완료!
